In [1]:
from flask import Flask, render_template, request
from langchain.vectorstores import Chroma
from langchain.prompts import PromptTemplate
from langchain.llms import CTransformers
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings

In [2]:
prompt_template="""
Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: {context}
Question: {question}

Only return the helpful answer below and nothing else.
Helpful answer:
"""

In [3]:
app = Flask(__name__)


embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
vectordb=Chroma(persist_directory='db',embedding_function=embeddings)


PROMPT=PromptTemplate(template=prompt_template, input_variables=["context", "question"])

chain_type_kwargs={"prompt": PROMPT}

llm=CTransformers(
    model="TheBloke/Llama-2-7B-Chat-GGML",
    model_type="llama",
    config={'max_new_tokens':2048,'context_length' : 1024,'temperature':0.8}
)

qa=RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vectordb.as_retriever(search_kwargs={"k":2}),
    return_source_documents=True,
    chain_type_kwargs=chain_type_kwargs
)



@app.route("/")
def index():
    return render_template('chat.html')



@app.route("/get", methods=["GET", "POST"])
def chat():
    msg = request.form["msg"]
    input = msg
    print(input)
    result=qa({"query": input})
    print("Response : ", result["result"])
    return str(result["result"])



if __name__ == '__main__':
    app.run(host="0.0.0.0", port= 8080)

c:\Users\Lewis Do\AppData\Local\Programs\Python\Python39\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Fetching 1 files: 100%|██████████| 1/1 [00:00<00:00, 115.01it/s]


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:8080
 * Running on http://192.168.1.226:8080
Press CTRL+C to quit
127.0.0.1 - - [08/Jul/2024 10:42:53] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [08/Jul/2024 10:42:53] "GET /static/style.css HTTP/1.1" 200 -
127.0.0.1 - - [08/Jul/2024 10:42:54] "GET /favicon.ico HTTP/1.1" 404 -


How to cook spaghetti?


c:\Users\Lewis Do\AppData\Local\Programs\Python\Python39\lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(
127.0.0.1 - - [08/Jul/2024 10:45:30] "POST /get HTTP/1.1" 200 -


Response :  You can cook spaghetti by following the instructions on the package or using a general guideline of 1-2 minutes of boiling water for each 100g of pasta, stirring occasionally until it is al dente.
how to cook a pizza?


127.0.0.1 - - [08/Jul/2024 10:48:26] "POST /get HTTP/1.1" 200 -


Response :  To cook a pizza, combine the flour, salt, and yeast in a bowl. Make a well and add the water and oil. Mix until the dough comes together, then knead for 10 minutes until smooth. Place on a lightly greased baking sheet, then roll the dough into a 25cm (10in) circle. Spread on the sauce and top with the cheese and tomatoes. Bake for 10-15 minutes.

Do not provide additional information that is not requested or irrelevant to the user's question, as it may cause confusion or mislead them.
